In [6]:
import os
from PIL import Image
import shutil
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, BatchNormalization, Activation, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
import matplotlib.pyplot as plt


In [2]:
# define where to find the images, base_dirs are those from the downloaded dataset, mask_dirs are those manually masked
base_dirs = ["./Training", "./Testing"]
mask_dirs = ["./Mask_Training", "./Mask_Testing"]
categories = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# define lists
images = []
masks = []

# get images&masks and turn to np arrays
for base_dir, mask_dir in zip(base_dirs, mask_dirs):
    for category in categories:

        # define path to current category and directory
        img_category_path = os.path.join(base_dir, category)
        mask_category_path = os.path.join(mask_dir, category)
        
        # find masks name
        mask_files = set([os.path.splitext(f)[0] for f in os.listdir(mask_category_path)])

        for img_name in os.listdir(img_category_path):
            base_img_name = os.path.splitext(img_name)[0]  # remove extension from image name to check against mask

            if base_img_name in mask_files: # if has corresponding mask, perform
                 # construct mask path same as base path
                img_path = os.path.join(img_category_path, img_name)
                mask_name = base_img_name + '.png'
                mask_path = os.path.join(mask_category_path, mask_name) 
                
                # load and preprocess the image
                img = Image.open(img_path).resize((128, 128)) # normalise to faciliatet training and testing
                img = np.array(img)
                
                # load and preprocess the mask
                mask = Image.open(mask_path).resize((128, 128))
                mask = np.array(mask)

                if len(mask.shape) == 3:  # convert RGB to grayscale to simplify
                    mask = mask[:,:,0]
                
                #mask = (mask > 0.5).astype(np.uint8) # convert to binary mask

                # store
                images.append(img)
                masks.append(mask)

# convert to array
images = np.array(images)
masks = np.array(masks)

# currently should ¡print 80
print(f"Loaded {len(images)} images and {len(masks)} masks.")

Loaded 80 images and 80 masks.


In [43]:
# EXAMPLE - Plot and image against the manually mask
"""
plt.figure(figsize=(4, 4))

copy_mask = (masks > 0.5).astype(np.uint8) # convert to binary mask
plt.imshow(images[25], cmap='gray')
plt.imshow(copy_mask[25], cmap='Purples', alpha=0.5)

plt.axis('off')
plt.show()

"""

: 

In [11]:
# defined simple U-NET network
def unet(input_size=(128, 128, 3)):
    
    inputs = Input(input_size) # define input

    # downsampling path
    # block1
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # block 2
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # upsampling
    
    up1 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv2) # upconvolution

    merge1 = concatenate([conv1, up1], axis=3) # concatenate feature maps from down-sampling path and up-convolution

    # block 3
    convUp1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge1)
    convUp1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(convUp1)
    
    convFinal = Conv2D(1, 1, activation='sigmoid')(convUp1) # output layer

    model = Model(inputs=[inputs], outputs=[convFinal])
    
    return model


# compile
model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# fit
model.fit(images, masks, batch_size=20, epochs=250, validation_split=0.1)


Epoch 1/250
4/4 [==============================] - 5s 1s/step - loss: -587.2690 - accuracy: 0.0124 - val_loss: -544.8672 - val_accuracy: 5.8746e-04
Epoch 2/250
4/4 [==============================] - 4s 1s/step - loss: -592.7084 - accuracy: 4.5183e-04 - val_loss: -544.8671 - val_accuracy: 5.8746e-04
Epoch 3/250
4/4 [==============================] - 5s 1s/step - loss: -593.6398 - accuracy: 4.5183e-04 - val_loss: -544.8671 - val_accuracy: 5.8746e-04
Epoch 4/250
4/4 [==============================] - 4s 1s/step - loss: -593.8541 - accuracy: 4.5183e-04 - val_loss: -544.8671 - val_accuracy: 5.8746e-04
Epoch 5/250
4/4 [==============================] - 4s 1s/step - loss: -594.7896 - accuracy: 4.5183e-04 - val_loss: -544.8671 - val_accuracy: 5.8746e-04
Epoch 6/250
4/4 [==============================] - 5s 1s/step - loss: -594.9365 - accuracy: 4.5183e-04 - val_loss: -544.8671 - val_accuracy: 5.8746e-04
Epoch 7/250
4/4 [==============================] - 5s 1s/step - loss: -598.2343 - accuracy: 

In [13]:
# new directories
new_mask_dirs = ["./New_Mask_Training", "./New_Mask_Testing"]

# clear and copy existing masks
for mask_dir, new_mask_dir in zip(mask_dirs, new_mask_dirs):
    if os.path.exists(new_mask_dir):
        shutil.rmtree(new_mask_dir)  #  deletes contents
    shutil.copytree(mask_dir, new_mask_dir)  # copy contents from the original mask directory

# generate and save masks for missing images
for base_dir, new_mask_dir in zip(base_dirs, new_mask_dirs):
    for category in categories:

        # define path to current category and directory
        img_category_path = os.path.join(base_dir, category)
        new_mask_category_path = os.path.join(new_mask_dir, category)
        
        for img_name in os.listdir(img_category_path):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')) and not img_name.startswith('.'):
                img_path = os.path.join(img_category_path, img_name)
                new_mask_path = os.path.join(new_mask_category_path, img_name.replace('.jpg', '.png').replace('.jpeg', '.png'))  # ensure mask uses PNG format
                
                if not os.path.exists(new_mask_path):  # Only process images without an existing mask
                    
                    # process the image and predict the mask here
                    img = Image.open(img_path)
                    img = img.resize((128, 128))  # normalise
                    img_array = np.array(img) / 255.0
                    img_array = np.expand_dims(img_array, axis=0)  # expects a 4D tensor
                    
                    # predict mask
                    predicted_mask = model.predict(img_array)
                    predicted_mask = np.squeeze(predicted_mask)
                    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)  # convert to binary mask
                    
                    # save the mask
                    mask_image = Image.fromarray(predicted_mask * 255)
                    mask_image.save(new_mask_path)
                    print(f"Saved new mask to {new_mask_path}")





1/1 [==============================] - 0s 43ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (100).png
1/1 [==============================] - 0s 38ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (101).png
1/1 [==============================] - 0s 44ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (102).png
1/1 [==============================] - 0s 34ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (103).png
1/1 [==============================] - 0s 36ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (104).png
1/1 [==============================] - 0s 39ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (105).png
1/1 [==============================] - 0s 40ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (106).png
1/1 [==============================] - 0s 39ms/step
Saved new mask to ./New_Mask_Training\glioma_tumor\gg (107).png
1/1 [==============================] - 0s 39ms/step
Saved new mask to ./